In [11]:
import keras
print(keras.__version__)

2.3.1


In [12]:
import tensorflow
print(tensorflow.__version__)

2.1.0


In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, History
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import glob

Using TensorFlow backend.


In [5]:
from tensorflow.keras.applications.vgg16 import VGG16

In [6]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2 as cv

In [8]:
import urllib

In [7]:
input_shape=(224,224,3)

In [8]:
model= Sequential()

model.add(Conv2D(filters=16, kernel_size=(5,5),activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(filters=16, kernel_size=(5,5),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32, kernel_size=(5,5),activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(5,5),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

In [9]:
model.compile(optimizer='RMSprop',loss='categorical_crossentropy',metrics=['Accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 16)      1216      
_________________________________________________________________
batch_normalization (BatchNo (None, 220, 220, 16)      64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 216, 216, 16)      6416      
_________________________________________________________________
batch_normalization_1 (Batch (None, 216, 216, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 108, 108, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 108, 108, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 104, 104, 32)      1

In [10]:
model1= VGG16(weights='imagenet',include_top=False, input_shape= (224,224,3))
model1.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
model1.save('vgg16.h5')

In [12]:
image_size=224

In [13]:
def prepare_dataset(path,label):
    x_train=[]
    y_train=[]
    all_images_path=glob.glob(path+'/*.jpeg')
    for img_path in all_images_path :
            img=load_img(img_path, target_size=(224,224))
            img=img_to_array(img)
            img=img/255.0
            x_train.append(img)
            y_train.append(label)
    return np.array(x_train),np.array(y_train)

In [14]:
paths=glob.glob(r'C:\Users\Kartikay\Desktop\DS Work\Datasets\Car Damage Detection\car-damage-detective-master\Data for images\data1a\training\*')
l=len(r'C:\Users\Kartikay\Desktop\DS Work\Datasets\Car Damage Detection\car-damage-detective-master\Data for images\data1a\training\*')
labels=[]
for path in paths:
    labels.append(path[l:])
    print(labels)

['0-damage']
['0-damage', '1-whole']


In [15]:
trainX_damage,trainY_damage= prepare_dataset(r"C:\Users\Kartikay\Desktop\DS Work\Datasets\Car Damage Detection\car-damage-detective-master\Data for images\data1a\training\00-damage",0)
trainX_whole,trainY_whole  = prepare_dataset(r"C:\Users\Kartikay\Desktop\DS Work\Datasets\Car Damage Detection\car-damage-detective-master\Data for images\data1a\training\01-whole",1)



In [16]:
print('train damage',     trainX_damage.shape, trainY_damage.shape)
print('train whole', trainX_whole.shape ,trainY_whole.shape)

train damage (920, 224, 224, 3) (920,)
train whole (128, 224, 224, 3) (128,)


In [17]:
x_train=np.concatenate((trainX_damage,trainX_whole),axis=0)
y_train=np.concatenate((trainY_damage,trainY_whole),axis=0)

In [18]:
print(x_train.shape)
print(y_train.shape)

(1048, 224, 224, 3)
(1048,)


In [19]:
testX_damage,testY_damage  = prepare_dataset(r"C:\Users\Kartikay\Desktop\DS Work\Datasets\Car Damage Detection\car-damage-detective-master\Data for images\data1a\validation\00-damage",0)
testX_whole,testY_whole  = prepare_dataset(r"C:\Users\Kartikay\Desktop\DS Work\Datasets\Car Damage Detection\car-damage-detective-master\Data for images\data1a\validation\01-whole",1)


In [20]:
x_test=np.concatenate((testX_damage,testX_whole),axis=0)
y_test=np.concatenate((testY_damage,testY_whole),axis=0)

In [21]:
print(x_test.shape)
print(y_test.shape)

(261, 224, 224, 3)
(261,)


In [22]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [23]:
from tensorflow.keras import layers
from tensorflow.keras import Model
for layer in model1.layers:
     layer.trainable = False

In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_accuracy', verbose=3,
    save_best_only=True, mode='auto', period=1)

In [25]:
# pretrained_model.summary()
last_layer = model1.get_layer('block5_pool')
print('last layer of vgg : output shape: ', last_layer.output_shape)
last_output= last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(2, activation='softmax')(x)           

model_vgg = Model(model1.input, x) 


model_vgg.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

history= model_vgg.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test))

last layer of vgg : output shape:  (None, 7, 7, 512)
Train on 1048 samples, validate on 261 samples
1048/1048 [==============================] - 626s 598ms/sample - loss: 0.6864 - acc: 0.8416 - val_loss: 0.2528 - val_acc: 0.9004
